# [[5,1,3]] code

This is our attempt at implementing the [[5,1,3]] code. The code works for an initial state of |0> however, is unable to adapt for the different applied unitaries. 

# Libraries

In [1]:
# Import PennyLane library for quantum circuit simulation.
import pennylane as qml

# Import NumPy for numerical operations and array handling.
import numpy as np

# Import matplotlib for plotting and visualizing results.
import matplotlib.pyplot as plt

# Unitary

In [2]:
# Set of 2-design sequences for single-qubit unitary 2-design.
single_qubit_two_design = [
    '',     # Identity.
    'Y',    # Apply RY with specific angle.
    'YA',   # RY followed by RZ(2π/3).
    'YB'    # RY followed by RZ(4π/3).
]

def apply_single_two_design(two_design_string, wires):
    """
    Applies a single-qubit 2-design sequence to the given wire.

    Args:
        two_design_string (str): A string of gates to apply.
        wires (int or list[int]): The qubit wire(s) to apply the gates to.
    """
    for gate in two_design_string:
        if gate == 'Y':
            qml.RY(2*np.arccos(1/np.sqrt(3)), wires=wires)
        elif gate == 'A':
            qml.RZ(2*np.pi/3, wires=wires)
        elif gate == 'B':
            qml.RZ(4*np.pi/3, wires=wires)

def apply_sequence(sequence, wires):
    """
    Applies a sequence of single-qubit 2-design operations.

    Args:
        sequence (list[str]): List of design strings to apply.
        wires (list[int]): Wires that match with the sequence list.
    """
    for two_design in sequence:
        apply_single_two_design(two_design, wires)

def apply_inverse_sequence(sequence, wires):
    """
    Applies the adjoint of a sequence of 2-design operations.

    Args:
        sequence (list[str]): List of design strings to invert.
        wires (list[int]): Wires that match with the sequence list.
    """
    for two_design in reversed(sequence):
        apply_single_two_design_adjoint(two_design, wires)

def apply_single_two_design_adjoint(two_design_string, wires):
    """
    Applies the adjoint of a single 2-design sequence.

    Args:
        two_design_string (str): String of gates to invert.
        wires (int or list[int]): The wire(s) to apply the adjoint gates to.
    """
    for gate in reversed(two_design_string):
        if gate == 'Y':
            qml.adjoint(qml.RY)(2 * np.arccos(1 / np.sqrt(3)), wires=wires)
        elif gate == 'A':
            qml.adjoint(qml.RZ)(2 * np.pi / 3, wires=wires)
        elif gate == 'B':
            qml.adjoint(qml.RZ)(4 * np.pi / 3, wires=wires)

# Noise Functions

In [3]:
def noise_model(p_ampdamp, wires):
    """
    Applies amplitude damping noise to the specified wires.

    Args:
        p_ampdamp (float): Probability of amplitude damping (range 0 to 1).
        wires (list[int]): List of wire indices to apply noise.
    """
    for wire in wires:
        qml.AmplitudeDamping(p_ampdamp, wires=wire)

# 5-qubit VGQEC Functions

In [4]:
# --- DEVICES ---
dev_sample = qml.device("default.mixed", wires=9, shots=1)
dev_analytic = qml.device("default.mixed", wires=9, shots=1000)

# --- ENCODER ---
def encoding_circuit(wires):
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    for wire in wires:
        qml.Hadamard(wires=wire)
    
    qml.IsingZZ(-np.pi/2, wires=[4, 0])
    qml.IsingZZ(-np.pi/2, wires=[0, 1])
    qml.IsingZZ(-np.pi/2, wires=[1, 2])
    qml.IsingZZ(-np.pi/2, wires=[2, 3])
    qml.IsingZZ(-np.pi/2, wires=[3, 4])

    
# --- SYNDROME MEASUREMENT ---
def measure_syndrome():
    
    # S1 = X Z Z X I
    qml.Hadamard(5)
    qml.Hadamard(0); qml.Hadamard(3)
    for i in [0, 1, 2, 3]:
        qml.CNOT(wires=[i, 5])
    qml.Hadamard(0); qml.Hadamard(3)

    # S2 = I X Z Z X
    qml.Hadamard(6)
    qml.Hadamard(1); qml.Hadamard(4)
    for i in [1, 2, 3, 4]:
        qml.CNOT(wires=[i, 6])
    qml.Hadamard(1); qml.Hadamard(4)

    # S3 = X I X Z Z
    qml.Hadamard(7)
    qml.Hadamard(0); qml.Hadamard(2)
    for i in [0, 2, 3, 4]:
        qml.CNOT(wires=[i, 7])
    qml.Hadamard(0); qml.Hadamard(2)

    # S4 = Z X I X Z
    qml.Hadamard(8)
    qml.Hadamard(1); qml.Hadamard(3)
    for i in [0, 1, 3, 4]:
        qml.CNOT(wires=[i, 8])
    qml.Hadamard(1); qml.Hadamard(3)
    

# --- CORRECTION MAPPING ---
def decode_and_correct(syndrome):
    syndrome_map = {
        (0, 0, 0, 0): None,
        (0, 0, 0, 1): (0, 'X'), (1, 0, 0, 0): (1, 'X'), 
        (1, 1, 0, 0): (2, 'X'), (0, 1, 1, 0): (3, 'X'), 
        (0, 0, 1, 1): (4, 'X'), (1, 0, 1, 0): (0, 'Z'), 
        (0, 1, 0, 1): (1, 'Z'), (0, 0, 1, 0): (2, 'Z'), 
        (1, 0, 0, 1): (3, 'Z'), (0, 1, 1, 1): (4, 'Z'),
        (1, 0, 1, 1): (0, 'Y'), (1, 1, 0, 1): (1, 'Y'), 
        (1, 1, 1, 0): (2, 'Y'), (1, 1, 1, 1): (3, 'Y'), 
        (0, 1, 0, 0): (4, 'Y')
    }
    key = tuple(syndrome)

    if key not in syndrome_map or syndrome_map[key] is None:
        return []
    
    qubit, op = syndrome_map[key]
    
    return [getattr(qml, f'Pauli{op}')(wires=qubit)]

                
@qml.qnode(dev_sample)
def apply_noise_and_measure(gamma):
    encoding_circuit(dev_sample.wires)
    noise_model(gamma, range(5))
    measure_syndrome()
    
    return qml.sample(wires=[5, 6, 7, 8])

@qml.qnode(dev_sample)
def apply_correction(correction_ops=[]):
    for op in correction_ops:
        op.queue()

    return qml.sample(wires=0)  # logical qubit


def compute_fc(fe, d=2):
    return (fe*(d + 1) - 1) / d

def call_plot(all_cost, all_noise):
    x = np.array(all_noise)
    y = np.array(all_cost)

    plt.figure(figsize=(7, 5))

    plt.plot(x, y, color='green', linestyle='-', label='Three-qubit VGQEC code')

    plt.xlabel(r'$\lambda$', fontsize=14)
    plt.ylabel('Channel Fidelity', fontsize=14)
    plt.title('Amplitude Damping Noise', fontsize=16)
    plt.xlim(0.0, 0.5)
    plt.ylim(0.75, 1.01)

    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.legend(fontsize=12, frameon=True, loc='lower left')

    plt.tight_layout()
    plt.show()

# Running [[5,1,3]] code for all noise values

In [ ]:
all_fc = []
all_gamma = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

iterations = 100

for gamma in all_gamma:

    total_fid = 0

    for i in range(iterations):

        syndrome = apply_noise_and_measure(gamma)

        correction_ops = decode_and_correct(list(syndrome))

        result = apply_correction(correction_ops)


        ent_fidelity = np.mean(result == 0)  # Measure probability of |0>
        total_fid += ent_fidelity


    all_fc.append(compute_fc(total_fid/iterations)) # Take avg of all iterations

    print("gamma = ", gamma, "; avg fid = ", compute_fc(total_fid/iterations), "\n")


call_plot(all_fc, all_gamma)